In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
!unzip /content/dataset.zip -d "/content/drive/MyDrive/CAR DENT PROJECT/Dataset/"

Archive:  /content/dataset.zip
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/41.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/06.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/dataset.xlsx  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/29.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/10.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/33.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/38.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/12.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/01.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/08.jpg  
  inflating: /content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/images/02.jpg  
  inflating: /content/drive/MyDrive

In [ ]:
BASE = "/content/drive/MyDrive/CAR DENT PROJECT/Dataset/Dataset/"

CSV_PATH = BASE + "dataset.csv"
IMG_DIR = BASE + "images/"
df = pd.read_csv(CSV_PATH)
print("Total samples:", len(df))

Total samples: 41


In [ ]:
df = df[["image", "depth"]]
df.head()

,image,depth
0,01.jpg,1.5
1,02.jpg,2.0
2,03.jpg,1.3
3,04.jpg,3.0
4,05.jpg,1.2


In [ ]:
class DentDataset(Dataset):
    def __init__(self, df, img_dir, targets, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image"]
        img_path = f"{self.img_dir}/{img_name}"

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)["image"]

        image = image.float()        # IMPORTANT FIX

        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return image, y

In [ ]:
train_tfms = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.6),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15,
                       rotate_limit=15, p=0.7),
    A.GaussianBlur(p=0.2),
    A.RandomShadow(p=0.3),
    A.RandomFog(p=0.2),
    A.Normalize(mean=(0.485,0.456,0.406),
                std=(0.229,0.224,0.225)),
    ToTensorV2()
])

valid_tfms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485,0.456,0.406),
                std=(0.229,0.224,0.225)),
    ToTensorV2()
])


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=59)

In [ ]:
scaler = StandardScaler()
scaler.fit(train_df[['depth']].values)   # fit only on train

train_targets = scaler.transform(train_df[['depth']].values)
val_targets   = scaler.transform(val_df[['depth']].values)

In [ ]:
import joblib
joblib.dump(scaler, "depth_scaler.pkl")

['depth_scaler.pkl']

In [ ]:
train_dataset = DentDataset(train_df, IMG_DIR, train_targets, train_tfms)
val_dataset   = DentDataset(val_df, IMG_DIR, val_targets, valid_tfms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
class DentRegressor(nn.Module):
    def __init__(self):
        super().__init__()

        self.backbone = models.resnet18(weights="IMAGENET1K_V1")

        # Freeze all layers
        for name, param in self.backbone.named_parameters():
            param.requires_grad = False

        # Unfreeze layer4 only
        for name, param in self.backbone.layer4.named_parameters():
            param.requires_grad = True

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()   # 512-d output

        self.regressor = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.10),

            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.05),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        feat = self.backbone(x)
        return self.regressor(feat)

In [ ]:
model = DentRegressor().cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_val = 1e9

for epoch in range(40):

    model.train()
    train_loss = 0
    for img, target in train_loader:
        img, target = img.cuda(), target.cuda()

        pred = model(img)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_loader)

    # -------- Validation --------
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for img, target in val_loader:
            img, target = img.cuda(), target.cuda()
            pred = model(img)
            loss = criterion(pred, target)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), "best_model.pth")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


Epoch 1 | Train: 1.0005 | Val: 2.4891
Epoch 2 | Train: 0.9041 | Val: 2.2772
Epoch 3 | Train: 0.8769 | Val: 2.2160
Epoch 4 | Train: 0.7498 | Val: 2.2701
Epoch 5 | Train: 0.6765 | Val: 2.2659
Epoch 6 | Train: 0.5885 | Val: 2.2651
Epoch 7 | Train: 0.4629 | Val: 2.2889
Epoch 8 | Train: 0.5069 | Val: 2.3292
Epoch 9 | Train: 0.5900 | Val: 2.4516
Epoch 10 | Train: 0.4952 | Val: 2.5412
Epoch 11 | Train: 0.3200 | Val: 2.7560
Epoch 12 | Train: 0.4601 | Val: 2.9236
Epoch 13 | Train: 0.4025 | Val: 3.2125
Epoch 14 | Train: 0.3734 | Val: 3.4037
Epoch 15 | Train: 0.4016 | Val: 3.4547
Epoch 16 | Train: 0.2529 | Val: 3.6577
Epoch 17 | Train: 0.2205 | Val: 3.6914
Epoch 18 | Train: 0.2672 | Val: 3.6377
Epoch 19 | Train: 0.3197 | Val: 3.5858
Epoch 20 | Train: 0.1516 | Val: 3.6387
Epoch 21 | Train: 0.2316 | Val: 3.6122
Epoch 22 | Train: 0.1985 | Val: 3.5514
Epoch 23 | Train: 0.1117 | Val: 3.4753
Epoch 24 | Train: 0.2566 | Val: 3.5518
Epoch 25 | Train: 0.3072 | Val: 3.6179
Epoch 26 | Train: 0.2615 | Val: 3.

In [ ]:
df

,image,depth
0,01.jpg,1.5
1,02.jpg,2.0
2,03.jpg,1.3
3,04.jpg,3.0
4,05.jpg,1.2
5,06.jpg,2.1
6,07.jpg,2.8
7,08.jpg,0.7
8,09.jpg,0.6
9,10.jpg,2.8


In [ ]:
val_df

,image,depth
36,37.jpg,2.5
32,33.jpg,3.5
27,28.jpg,1.3
39,40.jpg,2.1
24,25.jpg,2.2
38,39.jpg,3.3
40,41.jpg,3.7
34,35.jpg,3.0
21,22.jpg,3.0


In [ ]:
def predict(img_path):
    model.eval()

    # Load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply inference transforms
    img_tensor = valid_tfms(image=img)["image"].float().unsqueeze(0).cuda()

    # Predict
    with torch.no_grad():
        pred_scaled = model(img_tensor).cpu().numpy()[0]   # scaled output

    # Inverse transform to get real scale values
    pred_real = scaler.inverse_transform([pred_scaled])[0]

    return {
        "depth":  float(pred_real[0]),
    }

# Test
example = predict(IMG_DIR + df.iloc[15]["image"])
example

{'depth': 3.015297521495337}

In [ ]:
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for img, target in val_loader:
        img, target = img.cuda(), target.cuda()
        pred = model(img)
        all_preds.append(pred.cpu())
        all_targets.append(target.cpu())

# Concatenate all batches
all_preds = torch.cat(all_preds, dim=0)
all_targets = torch.cat(all_targets, dim=0)

# --- Normalized Metrics ---
mse_norm  = F.mse_loss(all_preds, all_targets).item()
mae_norm  = F.l1_loss(all_preds, all_targets).item()
rmse_norm = np.sqrt(mse_norm)

# --- Denormalize using scaler ---
# Make sure to convert tensors to numpy arrays before inverse_transform
all_preds_denorm   = scaler.inverse_transform(all_preds.numpy())
all_targets_denorm = scaler.inverse_transform(all_targets.numpy())

# Convert back to tensors for metric computation
all_preds_denorm   = torch.tensor(all_preds_denorm)
all_targets_denorm = torch.tensor(all_targets_denorm)

# --- Denormalized Metrics ---
mse_denorm  = F.mse_loss(all_preds_denorm, all_targets_denorm).item()
mae_denorm  = F.l1_loss(all_preds_denorm, all_targets_denorm).item()
rmse_denorm = np.sqrt(mse_denorm)

print("\n📊 VALIDATION METRICS")
print(f"Normalized -> MSE: {mse_norm:.4f}, MAE: {mae_norm:.4f}, RMSE: {rmse_norm:.4f}")
print(f"Denormalized -> MSE: {mse_denorm:.4f}, MAE: {mae_denorm:.4f}, RMSE: {rmse_denorm:.4f}")


📊 VALIDATION METRICS
Normalized -> MSE: 1.6135, MAE: 1.0580, RMSE: 1.2703
Denormalized -> MSE: 1.2070, MAE: 0.9150, RMSE: 1.0986
